<a href="https://colab.research.google.com/github/Jaredkmacho/LinearAlgebraNotebooks/blob/main/CrossProduct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross Product
Given two 3D vectors. `V` and `W` the cross product `V x W` is a vector orthogonal to both `V` and `W`. `V x W` direction will be determined by the right hand rule.

The magnitude of `V x W` is equal to the area of a parallelogram of base `||V||` and height `||W||sin(theta)` where `theta` is the angle between `V` and `W`.

`||V x W|| = ||V||||W||sin(theta)`








In [2]:
%%writefile linear_algebra.h

#ifndef LINEAR_ALGEBRA_H
#define LINEAR_ALGEBRA_H

#include <iostream>
#include <vector>
#include <cmath>

class Vector {
public:
    Vector();
    Vector(double *coordinates, size_t len); //Constructor
    Vector(const Vector &other); //Copy constructor
    void print();
    void print(const char *name);
    double Magnitude();
    Vector Direction();
    Vector ProjectTo(Vector& base);
    Vector Project(Vector& other);
    double Dot(const Vector &other);
    double Angle(Vector &other);
    Vector Cross(Vector &other);
    double ParallelogramArea(Vector &other);
    double TriangleArea(Vector &other);
    bool Parallel(Vector &other);
    bool Orthogonal(Vector &other);
    bool operator==(const Vector &other);
    Vector& operator+=(const Vector &other);
    Vector& operator-=(const Vector &other);
    Vector& operator*=(const Vector &other);
    Vector& operator*=(double x);
    

private:
    std::vector<double> coordinates_;
    size_t dimension_;
};

Vector operator+(Vector v1, const Vector &v2);
Vector operator-(Vector v1, const Vector &v2);
Vector operator*(Vector v1, const Vector &v2);
Vector operator*(Vector v, double x);


#endif /* LINEAR_ALGEBRA_H */

Overwriting linear_algebra.h


In [24]:
%%writefile linear_algebra.cpp

#include "linear_algebra.h"
Vector::Vector() {
    coordinates_ = {};
    dimension_ = 0;
}

Vector::Vector(double *coordinates, size_t len) {
    for(size_t i = 0; i < len; i++) {
        coordinates_.emplace_back(coordinates[i]);
    }
    dimension_ = len;
}

Vector::Vector(const Vector &other) {
    for(double v : other.coordinates_) {
        coordinates_.emplace_back(v);
    }
    dimension_ = other.dimension_;
}

void Vector::print() {
    std::cout << "Vector: [";
    //std::cout.precision(4);
    for(size_t i = 0; i < dimension_; i++) {
        std::cout <<coordinates_[i] << ((i == dimension_ - 1) ? "]" : ",");
    }
    std::cout << std::endl;
}

void Vector::print(const char *name) {
    std::cout << name << ": [";
    //std::cout.precision(4);
    for(size_t i = 0; i < dimension_; i++) {
        std::cout <<coordinates_[i] << ((i == dimension_ - 1) ? "]" : ",");
    }
    std::cout << std::endl;
}

double Vector::Magnitude() {
    double tmp = 0;
    for(double v: coordinates_) {
        tmp += std::pow(v, 2);
    }
    return std::sqrt(tmp);
}

Vector Vector::Direction() {
    Vector unit(*this);
    if(unit.Magnitude() == 0) {
        std::cout << "Zero magnitude vector!" << std::endl;
        return Vector();
    }
    double tmp = 1 / unit.Magnitude();
    return unit*=tmp;
}

Vector Vector::ProjectTo(Vector& base){
    auto Ub = base.Direction();
    auto dotres = this->Dot(Ub);
    auto Vpll = Ub * dotres;
    return Vpll;
}

Vector Vector::Project(Vector& other) {
    auto Ub = this->Direction();
    auto dotres = other.Dot(Ub);
    auto Vpll = Ub * dotres;

    return Vpll;
}

double Vector::Dot(const Vector &other) {
    double tmp = 0;
    if(dimension_ == other.dimension_) {
        for(size_t i = 0; i < dimension_; i++) {
            tmp += coordinates_[i] * other.coordinates_[i];
        }
        return tmp;
    } 
    std::cout << "Not equal sized vectors!" << std::endl;
    return -1;
}

double Vector::Angle(Vector &other) {
    if(dimension_ == other.dimension_) {
        return std::acos( Dot(other) / (Magnitude() * other.Magnitude()) );
    }
    std::cout << "Not equal sized vectors!" << std::endl;
    return -1;
}

//DONE: Generate the cross product vector of this vector and another vector. 
Vector Vector::Cross(Vector &other) {
    //DONE: Validate that the vectors have exactly 3 dimensions.
    if(dimension_ != 3 || other.dimension_ != 3) {
        std::cout << "Error: Operation with non 3D Vector" << std::endl;
        return Vector();
    }
    double x1, y1, z1, x2, y2, z2;
    double cross[3];
    x1 = coordinates_[0];
    y1 = coordinates_[1];
    z1 = coordinates_[2];
    x2 = other.coordinates_[0];
    y2 = other.coordinates_[1];
    z2 = other.coordinates_[2];

    cross[0] = (y1 * z2) - (y2 * z1);
    cross[1] = (x2 * z1) - (x1 * z2);
    cross[2] = (x1 * y2) - (x2 * y1);

    return Vector(cross, sizeof(cross) / sizeof(double));

}

// DONE: Calculate the parallelogram area with ||V x W||
double Vector::ParallelogramArea(Vector &other) {
    auto crossvec = Cross(other);
    return crossvec.Magnitude();
}

// DONE: Calculate the Triangle Area with ||V x W|| / 2
double Vector::TriangleArea(Vector &other) {
    return ParallelogramArea(other) / 2;
}

bool Vector::Parallel(Vector &other) {
    //TODO: Check weather the "other" vector is a scalar product of the "this" vector, also 0 * v1 and v1 * v1 are parallel.
    // is "other" equal to "this"
    if(*this == other) {
        return true;
    }
    // is "other" or "this" a 0 vector? 
    else if( this->Magnitude() == 0.0 || other.Magnitude() == 0.0) {
        return true;
    }
    else if(std::cos(this->Angle(other)) >= 0.99 || std::cos(this->Angle(other)) <= -0.99) {
        return true;
    } 
    return false;
}

bool Vector::Orthogonal(Vector &other) {
    //TODO: Check weather a dot product is equal to 0
    return Dot(other) <= 0.01 && Dot(other) >= -0.01;
}

bool Vector::operator==(const Vector &other) {
    return coordinates_ == other.coordinates_;
}

Vector& Vector::operator+=(const Vector &other) {
    if(dimension_ == other.dimension_) {
        for(size_t i = 0; i < dimension_; i++) {
            coordinates_[i] += other.coordinates_[i];
        }
    return *this;
    }
}

Vector operator+(Vector v1, const Vector &v2) {
    v1 += v2;
    return v1;
}

Vector& Vector::operator-=(const Vector &other) {
    if(dimension_ == other.dimension_) {
        for(size_t i = 0; i < dimension_; i++) {
            coordinates_[i] = coordinates_[i] - other.coordinates_[i];
        }
    return *this;
    }
}

Vector operator-(Vector v1, const Vector &v2) {
    v1 -= v2;
    return v1;
}

Vector& Vector::operator*=(const Vector &other) {
    if(dimension_ == other.dimension_) {
        for(size_t i = 0; i < dimension_; i++) {
            coordinates_[i] *= other.coordinates_[i];
        }
    return *this;
    }
}

Vector operator*(Vector v1, const Vector &v2) {
    v1 *= v2;
    return v1;
}

Vector& Vector::operator*=(double x) {
    for(size_t i = 0; i < dimension_; i++) {
        coordinates_[i] *= x;
    }
    return *this;
}

Vector operator*(Vector v, double x) {
    v *= x;
    return v;
}


Overwriting linear_algebra.cpp


In [25]:
%%writefile main.cpp

#include "linear_algebra.h"


int main() {
    //Example 1
    double coordinates1[] = {8.462, 7.893, -8.187}; // V
    double coordinates2[] = {6.984, -5.975, 4.778}; // W
    //Example 2
    double coordinates3[] = {-8.987, -9.838, 5.031};  // V
    double coordinates4[] = {-4.268, -1.861, -8.866}; // W
    //Example 3
    double coordinates5[] = {1.5, 9.547, 3.691};    // V
    double coordinates6[] = {-6.007, 0.124, 5.772}; // W

    Vector vector1(coordinates1, sizeof(coordinates1) / sizeof(double));
    Vector vector2(coordinates2, sizeof(coordinates2) / sizeof(double));
    Vector vector3(coordinates3, sizeof(coordinates3) / sizeof(double));
    Vector vector4(coordinates4, sizeof(coordinates4) / sizeof(double));
    Vector vector5(coordinates5, sizeof(coordinates5) / sizeof(double));
    Vector vector6(coordinates6, sizeof(coordinates6) / sizeof(double));

    std::cout << "---------- Cross Product ---------" << std::endl;
    std::cout << "\nExample 1" << std::endl;
    vector1.print("V");
    vector2.print("W");
    auto crossVector = vector1.Cross(vector2);
    crossVector.print("V x W");

    std::cout << "\nExample 2" << std::endl;
    vector3.print("V");
    vector4.print("W");
    std::cout << "Area of parallelogram spanned by V and W: " << vector3.ParallelogramArea(vector4) << std::endl;;

    std::cout << "\nExample 3" << std::endl;
    vector5.print("V");
    vector6.print("W");
    std::cout << "Area of traingle spanned by V and W: " << vector5.TriangleArea(vector6) << std::endl;

}

Overwriting main.cpp


In [26]:
%%script bash
g++ linear_algebra.cpp main.cpp -std=c++11 -o linal

In [27]:
!./linal

---------- Cross Product ---------

Example 1
V: [8.462,7.893,-8.187]
W: [6.984,-5.975,4.778]
V x W: [-11.2046,-97.6094,-105.685]

Example 2
V: [-8.987,-9.838,5.031]
W: [-4.268,-1.861,-8.866]
Area of parallelogram spanned by V and W: 142.122

Example 3
V: [1.5,9.547,3.691]
W: [-6.007,0.124,5.772]
Area of traingle spanned by V and W: 42.5649
